# Convolutional Neural Network (Multi Image Classification)

### This CNN is used for multi-image classification and is trained on a dataset that contains 10 different image classes, which are as follows:
0: Airplanes

1: Automobiles

2: Birds

3: Cats

4: Deers

5: Dogs

6: Frogs

7: Horses

8: Ships

9: Trucks

### The dataset used is The CIFAR-10 dataset, referenced in the bottom. Each batch file of the training dataset is a Python 'pickled' object, and hence, will require the 'unpickle' function to open the file. The data consists of 6 batches of images, each batch contains 10000 image dictionaries, which consistsof 1000 image dictionaries per class (10 class of 1000 images each). The image dictionary has a pixel array (with a key [b'data']) and a corresponding label (key [b'labels']), all valued between 0 to 9 for the 10 potential classes. We will combine 5 of the batches to form the training data & leave the 6th for the testing data. We will use the Tensorflow library, as well as Keras, Numpy & Pandas for all the data preprocessing functions

## 1. Data Preprocessing

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
dataset1 = unpickle('data_batch_1')
dataset2 = unpickle('data_batch_2')
dataset3 = unpickle('data_batch_3')
dataset4 = unpickle('data_batch_4')
dataset5 = unpickle('data_batch_5')
testset = unpickle('test_batch')

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,# to normalise pixel values (make it all between 0-1)
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
train_data_1 = np.array(dataset1[b'data'], dtype="float")
train_data_2 = np.array(dataset2[b'data'], dtype="float")
train_data_3 = np.array(dataset3[b'data'], dtype="float")
train_data_4 = np.array(dataset4[b'data'], dtype="float")
train_data_5 = np.array(dataset5[b'data'], dtype="float")
test_data = np.array(testset[b'data'], dtype="float")

train_data = np.concatenate((train_data_1,train_data_2,train_data_3,train_data_4,train_data_5),axis=0)

### Each image array is an array of 3072 pixel values, where the first 1024 contain the red channel values,  the second 1024 contain the green channel values and the third 1024 contain the blue channel values. In order to train our model on the data, we need to change the image pixel array format so that array is a list of pixels and each pixel is of this format: pixal = [redValue, greenValue, blueValue]. We will also need to reshape the array so that it is compatible with our model (reshape it into a multidimensional array). In order to do so, we will use the changeFormat Function.

In [9]:
def changeFormat(train_data):
    new_arr = []
    count = 0
    for image in train_data:
        for i in range(0,len(image)-2048):
            pixel_arr = []
            pixel_arr.append(image[i])
            pixel_arr.append(image[i+1024])
            pixel_arr.append(image[i+2048])  
            new_arr.append(pixel_arr)
    return new_arr

In [10]:
modified_array = changeFormat(train_data)

train_data_modified = np.reshape(modified_array, (50000, 32, 32, 3))

[[59.0, 62.0, 63.0],
 [43.0, 46.0, 45.0],
 [50.0, 48.0, 43.0],
 [68.0, 54.0, 42.0],
 [98.0, 73.0, 52.0],
 [119.0, 91.0, 63.0],
 [139.0, 107.0, 75.0],
 [145.0, 110.0, 80.0],
 [149.0, 117.0, 89.0],
 [149.0, 120.0, 93.0],
 [131.0, 103.0, 77.0],
 [125.0, 99.0, 76.0],
 [142.0, 115.0, 91.0],
 [144.0, 112.0, 86.0],
 [137.0, 105.0, 79.0],
 [129.0, 97.0, 71.0],
 [137.0, 106.0, 79.0],
 [134.0, 106.0, 76.0],
 [124.0, 97.0, 64.0],
 [139.0, 113.0, 78.0],
 [139.0, 112.0, 75.0],
 [133.0, 105.0, 69.0],
 [136.0, 105.0, 74.0],
 [139.0, 108.0, 77.0],
 [152.0, 120.0, 89.0],
 [163.0, 131.0, 100.0],
 [168.0, 136.0, 108.0],
 [159.0, 129.0, 102.0],
 [158.0, 130.0, 104.0],
 [158.0, 132.0, 108.0],
 [152.0, 125.0, 102.0],
 [148.0, 124.0, 103.0],
 [16.0, 20.0, 20.0],
 [0.0, 0.0, 0.0],
 [18.0, 8.0, 0.0],
 [51.0, 27.0, 8.0],
 [88.0, 51.0, 21.0],
 [120.0, 82.0, 43.0],
 [128.0, 89.0, 45.0],
 [127.0, 86.0, 44.0],
 [126.0, 87.0, 50.0],
 [116.0, 79.0, 44.0],
 [106.0, 70.0, 37.0],
 [101.0, 67.0, 35.0],
 [105.0, 70.0, 36.

In [12]:
train_label_1 = np.array(dataset1[b'labels'], dtype="float")
train_label_2 = np.array(dataset2[b'labels'], dtype="float")
train_label_3 = np.array(dataset3[b'labels'], dtype="float")
train_label_4 = np.array(dataset4[b'labels'], dtype="float")
train_label_5 = np.array(dataset5[b'labels'], dtype="float")

train_labels=[*train_label_1, *train_label_2, *train_label_3, *train_label_4, *train_label_5]

In [16]:
from sklearn.model_selection import train_test_split
(trainX, valX, trainY, valY) = train_test_split(train_data_modified, train_labels, test_size=0.20, random_state=42)

batch_size = 32
training_dataset = train_datagen.flow(trainX, trainY, batch_size=batch_size)

In [19]:
modified_test_array = changeFormat(test_data)
        
test_data_modified = np.reshape(modified_test_array, (10000, 32, 32, 3))

test_dataset = test_datagen.flow(test_data_modified, batch_size=1)

## 2. Building & compiling the CNN

In [21]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[32, 32, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

cnn.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [22]:
cnn.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [23]:
test_y = testset[b'labels']

In [24]:
cnn.fit(x = training_dataset, validation_data = (test_dataset), epochs = 80)

Epoch 1/80
1250/1250 [==============================] - 24s 18ms/step - loss: 1.5857 - accuracy: 0.4287 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/80
1250/1250 [==============================] - 24s 19ms/step - loss: 1.2760 - accuracy: 0.5508 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/80
1250/1250 [==============================] - 24s 19ms/step - loss: 1.1512 - accuracy: 0.5956 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/80
1250/1250 [==============================] - 24s 20ms/step - loss: 1.0704 - accuracy: 0.6267 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/80
1250/1250 [==============================] - 24s 20ms/step - loss: 1.0132 - accuracy: 0.6458 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/80
1250/1250 [==============================] - 27s 22ms/step - loss: 0.9694 - accuracy: 0.6623 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/80
1250/1250 [==============================] - 28s 22ms/step - loss

Epoch 54/80
1250/1250 [==============================] - 25s 20ms/step - loss: 0.4986 - accuracy: 0.8224 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 55/80
1250/1250 [==============================] - 24s 19ms/step - loss: 0.4972 - accuracy: 0.8242 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 56/80
1250/1250 [==============================] - 40s 32ms/step - loss: 0.5029 - accuracy: 0.8224 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 57/80
1250/1250 [==============================] - 22s 18ms/step - loss: 0.4941 - accuracy: 0.8248 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 58/80
1250/1250 [==============================] - 22s 17ms/step - loss: 0.4915 - accuracy: 0.8231 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 59/80
1250/1250 [==============================] - 26s 21ms/step - loss: 0.4887 - accuracy: 0.8271 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 60/80
1250/1250 [==============================] - 27s 22ms/step

## 3. Making Predictions to test the model

In [26]:
from keras.preprocessing import image
test_image = image.load_img('cat-image.jpeg', target_size = (32, 32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

### The predict function returns an array of length 10, with each entry corresponding to the probability the image is the respective class (there are 10 potential classes an image can be). I have made the 'returnIndex' function that will return the index of the largest probability, returning the most possible class the image is.

In [50]:
def returnIndex(arr):
    maxval = arr[0]
    index = 0
    for i in range(1,len(arr)):
        if arr[i] > maxval:
            maxval = arr[i]
            index = i
    return index

In [28]:
prediction = cnn.predict(test_image/255.0)
predictionIndex1 = returnIndex(prediction[0])
predictionIndex1

1/1 [==============================] - 0s 266ms/step


In [37]:
from keras.preprocessing import image
test_image2 = image.load_img('frog-image.jpeg', target_size = (32, 32))
test_image2 = image.img_to_array(test_image2)
test_image2 = np.expand_dims(test_image2, axis = 0)
prediction2 = cnn.predict(test_image2/255.0)
predictionIndex2 = returnIndex(prediction2[0])
predictionIndex2

In [41]:
from keras.preprocessing import image
test_image3 = image.load_img('airplane-image.jpeg', target_size = (32, 32))
test_image3 = image.img_to_array(test_image3)
test_image3 = np.expand_dims(test_image3, axis = 0)
prediction3 = cnn.predict(test_image3/255.0)
predictionIndex = returnIndex(prediction3[0])
predictionIndex

## Reference

### This tech report (Chapter 3) describes the dataset and the methodology followed when collecting it in much greater detail. Please cite it if you intend to use this dataset. 

### https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf  Alex Krizhevsky, 2009.

### If you wish to run this model on your machine, you can find the training and test data (data_batch_1 - data_batch_5 and test_batch) in the link below.

### https://www.cs.toronto.edu/~kriz/cifar.html